In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from os import listdir, makedirs
import matplotlib.pyplot as plt
from keras import layers, models, optimizers, regularizers, losses
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.preprocessing import image
from keras.datasets import mnist
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')
print(os.listdir("/home/yaagocruuz/Documentos/git/Topicos3/Trabalho Final/"))

/home/yaagocruuz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['retrain.py', 'example_code', '.ipynb_checkpoints', 'dataset', 'Trabalho Final.ipynb']


In [2]:
RESOLUTION = 60
BATCH_SIZE = 16


data_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2,
                                  horizontal_flip=True, validation_split=0.20)

train_generator = data_datagen.flow_from_directory(
        "dataset/",
        classes=['Caroline_Kennedy', 'Edward_Moore_Kennedy', 'Eunice_Kennedy_Shriver', 'Jacqueline_Lee_Bouvier', 
                 'Jean_Kennedy Smith', 'John_F_Kennedy_Jr', 'John_Fitzgerald_Kennedy', 
                 'Joseph_P_Kennedy_Sr', 'Robert_Francis_Kennedy', 'Rose_Elizabeth_Fitzgerald', 
                 'Rosemary_Kenndy', 'Outros'],
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical', subset="training")

val_generator = data_datagen.flow_from_directory(
        "dataset/",
        classes=['Caroline_Kennedy', 'Edward_Moore_Kennedy', 'Eunice_Kennedy_Shriver', 'Jacqueline_Lee_Bouvier', 
                 'Jean_Kennedy Smith', 'John_F_Kennedy_Jr', 'John_Fitzgerald_Kennedy', 
                 'Joseph_P_Kennedy_Sr', 'Robert_Francis_Kennedy', 'Rose_Elizabeth_Fitzgerald', 
                 'Rosemary_Kenndy', 'Outros'],
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical', subset="validation")

Found 281 images belonging to 12 classes.
Found 66 images belonging to 12 classes.


In [19]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 25s 0us/step


In [20]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(12, activation='softmax')(x)

In [21]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [23]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [25]:
# train the model on the new data for a few epochs
model.fit_generator(generator=train_generator, epochs=30, validation_data=val_generator)

Epoch 1/30


AbortedError: Compute received an exception:Status: 3, message: could not initialize a memory descriptor, in file tensorflow/core/kernels/mkl_maxpooling_op.cc:578
	 [[Node: max_pooling2d_36/MaxPool = _MklMaxPool[T=DT_FLOAT, _kernel="MklOp", data_format="NHWC", ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 2, 2, 1], workspace_enabled=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](mixed7_8/concat, mixed7_8/concat:1)]]

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)